In [2]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats.mstats import winsorize
from google.colab import drive, files
import io

# CountVectorizer

## 5. Stop words, lematizing, stemming

### Cargar datos y dividir dataset

In [3]:
from tensorflow.random import set_seed
set_seed(234730)

In [5]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test_without_label.csv')

In [6]:
train['id'] = train['id'].astype(str)
train['tweet'] = train['tweet'].astype(str)
train['label'] = train['label'].astype(str)

In [7]:
train.drop(['id'], axis=1)
train.set_index("id", inplace = True)

In [8]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
# parseo de category
train[['label']] = train[['label']].apply(lambda col: label_encoder.fit_transform(col))

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_validation, Y_train, Y_validation = train_test_split(
    train.drop('label', axis=1),
    train['label'],
    test_size=(1.0/3), random_state=42)

### Stop words y lemmatize

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import WordNetLemmatizer
import nltk

nltk.download('wordnet')
Lemmatizer = WordNetLemmatizer()
#analyzer = CountVectorizer().build_analyzer()

from zipfile import ZipFile
file_loc = '/root/nltk_data/corpora/wordnet.zip'
with ZipFile(file_loc, 'r') as z:
  z.extractall('/root/nltk_data/corpora/')

[nltk_data] Downloading package wordnet to /root/nltk_data...


### Stopwords, lemmatize, punctuacion
En las stop words vamos a borrar los @

In [63]:
#All_punct = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
All_punct = '''!()-[]{};:'"\,<>./?#%^&*_~'''
#CountVectorizer con stopwords de Natural Language Toolkit
nltk.download('stopwords')

def is_Not_Link(word):
  return word[0:4] != "http"

def remove_Punctuation(doc_split):
  for i in range(len(doc_split)):
    word = doc_split[i];
    if is_Not_Link(word):
      for elements in word:
        if elements in All_punct:
          doc_split[i] = word.replace(elements, "")
  return doc_split

def remove_Stopwords(doc_split):
  doc_with_Stopwords = doc_split.copy();
  i = 0
  while i < len(doc_split):
    word = doc_split[i];
    if word in nltk.corpus.stopwords.words('english'):
      doc_with_Stopwords.remove(word);
    i = i + 1
  return doc_with_Stopwords

def new_analyzer(doc):
  doc_split = doc.split();
  doc_split = remove_Punctuation(doc_split);
  doc_split = remove_Stopwords(doc_split);
  return (Lemmatizer.lemmatize(w.lower()) for w in doc_split);





nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [ ]:
print(nltk.corpus.stopwords.words('english'))

### Fit

In [64]:
vectorizer_stop_words_stemmed = CountVectorizer(analyzer=new_analyzer, max_features=5000)
bag_of_words = vectorizer_stop_words_stemmed.fit(X_train["tweet"])

print(bag_of_words.get_feature_names())

['', '#coronacheck', '#coronavirus', '#coronavirusupdates', '#covid', '#covid-19', '#covid19', '#covid19associated', '#covid19like', '#covid19nigeria', '#dyk', '#factcheck', '#gujarat', '#hcps', '#indiafightscorona', '#kayburley', '#sepsis', '#socialdistancing', '$100', '$50', '&amp', '&gt&gt&gt', '(#pmsby', '(1.6%', '(1.7%', '(1.8%', '(1/2', '(2/4', '(21.9%', '(3/4', '(4025079', '(act', '(cfr', '(coronavirus', '(covid-19', '(covid-19)', '(ie', '(india', '(pmjjby', '(ppe)', '(tpm', '(who', '+', '+1', '0', '01', '02', '04', '05', '06', '0800', '0930', '1', '1%', '10', '100', '1000', '1000+', '10000', '100000', '100000+', '1000190000', '100k', '101', '102', '1040', '105', '1054', '107', '10k', '10pm', '10th', '11', '1100', '110000', '111', '1130', '114', '115', '115000', '1154', '1155pm', '116', '117', '1170', '118', '1184', '119', '11th', '12', '120', '120000', '1206', '1217', '1219', '123', '128', '12th', '13', '130', '130000', '134', '13th', '14', '140', '1455', '1481', '14th', '15', 

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


### Transform

In [65]:
bag_of_words = vectorizer_stop_words_stemmed.transform(X_train["tweet"])
bag_of_words = pd.DataFrame(bag_of_words.toarray(), columns = vectorizer_stop_words_stemmed.get_feature_names())

In [23]:
bag_of_words.shape[1]

5000

# Red Neuronal

In [24]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.layers import LeakyReLU

In [46]:
adam_optimizer='Adam'
loss='binary_crossentropy'
metric='accuracy'

model = Sequential()
model.add(Dense(20, input_shape=(bag_of_words.shape[1],)))
model.add(Activation(LeakyReLU(alpha=0.3)))
model.add(Dense(30, use_bias=True))
model.add(Activation(LeakyReLU(alpha=0.3)))
model.add(Dense(1, use_bias=True))
model.add(Activation('sigmoid'))
model.compile(loss=loss, optimizer=adam_optimizer, metrics=[metric])

In [ ]:
X_train_array = np.asarray(bag_of_words)
Y_train_array = np.asarray(Y_train)

training = model.fit(X_train_array, Y_train_array, epochs=200, batch_size=100, validation_split=0.2)

In [53]:
import tensorflow.keras as keras

model.save("basic_model")
saved_model = keras.models.load_model("basic_model")

In [71]:
X_Validation_Bag = vectorizer_stop_words_stemmed.transform(X_validation["tweet"])
X_Validation_Bag_df = pd.DataFrame(X_Validation_Bag.toarray(), columns = vectorizer_stop_words_stemmed.get_feature_names())

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [72]:
X_Validation_Bag_df_array = np.asarray(X_Validation_Bag_df)
Y_Validation_Predict = saved_model.predict(X_Validation_Bag_df_array)

90/90 [==============================] - 0s 2ms/step


In [ ]:
Y_Validation_Predict

In [56]:
def transformPrediction(df, predNumber, predLabel):
  copy = df.copy()
  for i in range(len(copy)):
    if copy[i][0] > 0.5:
      predNumber.append([1])
      predLabel.append(["real"])
    else:
      predNumber.append([0])
      predLabel.append(["fake"])


In [73]:
predValidationNumbers = []
predValidationLabels = []
transformPrediction(Y_Validation_Predict, predValidationNumbers, predValidationLabels)

In [74]:
validationNumbers = []
validationLabels = []
copy = np.asarray(Y_validation).copy()
for i in range(len(copy)):
  if copy[i] > 0.5:
    validationNumbers.append([1])
    validationLabels.append(["real"])
  else:
    validationNumbers.append([0])
    validationLabels.append(["fake"])

In [41]:
np.asarray(Y_validation)

array([1, 1, 1, ..., 0, 1, 0])

In [76]:
from sklearn.metrics import accuracy_score
print(accuracy_score(predValidationLabels,validationLabels))

0.5073580939032937


# K fold

In [27]:
from sklearn.metrics import accuracy_score

In [28]:
import math
k = 5

def k_folds(X, y, k):
  assert(len(X) == len(y))
  folds_X = []
  folds_y = []
  initial_pos = 0

  for i in range(k):
    to_pos = min(math.ceil(len(X)/k)*(i+1), len(X))
    
    x_fold = X[initial_pos:to_pos]
    y_fold = y[initial_pos:to_pos]

    folds_X.append(x_fold)
    folds_y.append(y_fold)
    initial_pos = to_pos

  return folds_X, folds_y

In [29]:
def evaluate(model, X_to_eval, Y_to_eval):
    Y_Prediction = model.predict(np.asarray(X_to_eval))
    numbers = []
    prediction_labels = []
    transformPrediction(Y_Prediction, numbers, prediction_labels)

    validationLabels = []
    copy = np.asarray(Y_to_eval).copy()
    for i in range(len(copy)):
      if copy[i] > 0.5:
        validationLabels.append(["real"])
      else:
        validationLabels.append(["fake"])

    return accuracy_score(prediction_labels, validationLabels)

In [30]:
def k_fold_cross_val(folds_X, folds_y, model):
  assert(len(folds_X) == len(folds_y))
  X_to_train = []
  y_to_train = []
  X_to_eval = 0
  y_to_eval = 0
  scores = []
  for i in range(len(folds_X)):
    X_to_train = []
    y_to_train = []
    for j in range(len(folds_X)):
      if  i == j:
        X_to_eval = folds_X[i]
        y_to_eval = folds_y[i]
      else:
        X_to_train.append(folds_X[j])
        y_to_train.append(folds_y[j])
    
    X_train = np.concatenate(X_to_train)
    y_train = np.concatenate(y_to_train)
    model.fit(X_train,y_train, epochs=200, batch_size=100, validation_split=0.2)
    score = evaluate(model, X_to_eval, y_to_eval)
    scores.append(score)
  return np.mean(np.array(scores)), scores

In [66]:
folds_X, folds_y = k_folds(np.asarray(bag_of_words), np.asarray(Y_train), 5)


In [ ]:
k_fold_result_mean, k_fold_result_arr = k_fold_cross_val(folds_X, folds_y, model)

In [68]:
k_fold_result_mean

0.9580845124177051

In [69]:
k_fold_result_arr

[0.9010507880910683,
 0.978108581436077,
 0.9956217162872154,
 1.0,
 0.9156414762741653]

# Test

In [49]:
test['id'].astype('str')
test['tweet'].astype('str')

test_bag = vectorizer_stop_words_stemmed.transform(test["tweet"])
test_bag = pd.DataFrame(test_bag.toarray(), columns = vectorizer_stop_words_stemmed.get_feature_names())

test_bag_array = np.asarray(test_bag)
test_prediction = saved_model.predict(test_bag_array)

copy = test_prediction.copy()
predNumber = [];
predLabel = [];
for i in range(len(copy)):
  if copy[i][0] > 0.5:
    predNumber.append([1])
    predLabel.append(["real"])
  else:
    predNumber.append([0])
    predLabel.append(["fake"])

prediction = pd.DataFrame(predLabel, columns=['label'])
prediction.index += 1
prediction = prediction.to_csv('prediction.csv')

KeyboardInterrupt: ignored

In [ ]:
from google.colab import files

files.download('prediction.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>